In [1]:
import math
import numpy as np
from PIL import Image
import cv2
import skimage

In [2]:
import os

# Path to the directory you want to list
dataset_path = './srgan_dataset'

# List all files and folders in the directory
files = os.listdir(dataset_path)

# Print the list of files
for file in files:
    print(file)

change_names.ipynb
test
test_HR
test_LR
test_SR
test_SR_content
train_HR
train_LR
train_SR
train_SR_content
valid_HR
valid_LR
valid_SR
valid_SR_content


In [12]:
mode = "test"
hr_name_path = os.path.join(dataset_path,mode+"_HR")
lr_name_path = os.path.join(dataset_path,mode+"_LR")
sr_name_path = os.path.join(dataset_path,mode+"_SR")
image_names = list(os.listdir(hr_name_path))

In [97]:
n = 40
hr_sample = Image.open(os.path.join(hr_name_path,image_names[n]))
hr_sample = np.transpose(np.array(hr_sample,dtype=np.float64),(2,0,1))
lr_sample = Image.open(os.path.join(lr_name_path,image_names[n]))
lr_sample = np.transpose(np.array(lr_sample,dtype=np.float64),(2,0,1))
sr_sample = Image.open(os.path.join(sr_name_path,image_names[n]))
sr_sample = np.transpose(np.array(sr_sample,dtype=np.float64),(2,0,1))

In [3]:
def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

In [63]:
# print(calculate_ssim(hr_sample,lr_sample),calculate_psnr(hr_sample,lr_sample),"   ",skimage.metrics.structural_similarity(hr_sample,lr_sample,data_range=255),skimage.metrics.peak_signal_noise_ratio(hr_sample,lr_sample,data_range=255))
print(calculate_psnr(hr_sample,lr_sample),calculate_psnr(hr_sample,sr_sample))

19.371847863357083 27.066042506251744


In [4]:
def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

In [90]:
print(calculate_ssim(hr_sample,lr_sample),calculate_ssim(hr_sample,sr_sample))

None None


In [6]:
import torch  
import torch.nn.functional as F 
import cv2

In [7]:
def gaussian(window_size, sigma):
    """
    Generates a list of Tensor values drawn from a gaussian distribution with standard
    diviation = sigma and sum of all elements = 1.

    Length of list = window_size
    """    
    gauss =  torch.Tensor([math.exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()

gauss_dis = gaussian(11, 1.5)
print("Distribution: ", gauss_dis)
print("Sum of Gauss Distribution:", torch.sum(gauss_dis))

Distribution:  tensor([0.0010, 0.0076, 0.0360, 0.1094, 0.2130, 0.2660, 0.2130, 0.1094, 0.0360,
        0.0076, 0.0010])
Sum of Gauss Distribution: tensor(1.)


In [8]:
def create_window(window_size, channel=1):

    # Generate an 1D tensor containing values sampled from a gaussian distribution
    _1d_window = gaussian(window_size=window_size, sigma=1.5).unsqueeze(1)
    
    # Converting to 2D  
    _2d_window = _1d_window.mm(_1d_window.t()).float().unsqueeze(0).unsqueeze(0)
     
    window = torch.Tensor(_2d_window.expand(channel, 1, window_size, window_size).contiguous())

    return window

window = create_window(11, 3)
print("Shape of gaussian window:", window.shape)

Shape of gaussian window: torch.Size([3, 1, 11, 11])


In [9]:
def ssim(img1, img2, val_range, window_size=11, window=None, size_average=True, full=False):

    L = val_range # L is the dynamic range of the pixel values (255 for 8-bit grayscale images),

    pad = window_size // 2
    
    try:
        _, channels, height, width = img1.size()
    except:
        channels, height, width = img1.size()

    # if window is not provided, init one
    if window is None: 
        real_size = min(window_size, height, width) # window should be atleast 11x11 
        window = create_window(real_size, channel=channels).to(img1.device)
    
    # calculating the mu parameter (locally) for both images using a gaussian filter 
    # calculates the luminosity params
    mu1 = F.conv2d(img1, window, padding=pad, groups=channels)
    mu2 = F.conv2d(img2, window, padding=pad, groups=channels)
    
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2 
    mu12 = mu1 * mu2

    # now we calculate the sigma square parameter
    # Sigma deals with the contrast component 
    sigma1_sq = F.conv2d(img1 * img1, window, padding=pad, groups=channels) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=pad, groups=channels) - mu2_sq
    sigma12 =  F.conv2d(img1 * img2, window, padding=pad, groups=channels) - mu12

    # Some constants for stability 
    C1 = (0.01 ) ** 2  # NOTE: Removed L from here (ref PT implementation)
    C2 = (0.03 ) ** 2 

    contrast_metric = (2.0 * sigma12 + C2) / (sigma1_sq + sigma2_sq + C2)
    contrast_metric = torch.mean(contrast_metric)

    numerator1 = 2 * mu12 + C1  
    numerator2 = 2 * sigma12 + C2
    denominator1 = mu1_sq + mu2_sq + C1 
    denominator2 = sigma1_sq + sigma2_sq + C2

    ssim_score = (numerator1 * numerator2) / (denominator1 * denominator2)

    if size_average:
        ret = ssim_score.mean() 
    else: 
        ret = ssim_score.mean(1).mean(1).mean(1)
    
    if full:
        return ret, contrast_metric
    
    return ret

In [10]:
tensorify = lambda x: torch.Tensor(x.transpose((2, 0, 1))).unsqueeze(0).float().div(255.0)

In [98]:
hr_tens = tensorify(hr_sample)
sr_tens = tensorify(sr_sample)
lr_tens = tensorify(lr_sample)

In [104]:
ssim(hr_tens,lr_tens,val_range=255).item()

0.9661716818809509

In [12]:
modes = ["test","train","valid"]

for mode in modes:
    hr_name_path = os.path.join(dataset_path,mode+"_HR")
    lr_name_path = os.path.join(dataset_path,mode+"_LR")
    sr_name_path = os.path.join(dataset_path,mode+"_SR")
    src_name_path = os.path.join(dataset_path,mode+"_SR_content")
    image_names = list(os.listdir(hr_name_path))
    n = len(image_names)
    lr_psnr = []
    sr_psnr = []
    src_psnr = []
    lr_ssim = []
    sr_ssim = []
    src_ssim = []

    for i in range(n):
        hr_sample = Image.open(os.path.join(hr_name_path,image_names[i]))
        hr_tens = tensorify(np.array(hr_sample))
        hr_sample = np.transpose(np.array(hr_sample,dtype=np.float64),(2,0,1))
        lr_sample = Image.open(os.path.join(lr_name_path,image_names[i]))
        lr_tens = tensorify(np.array(lr_sample))
        lr_sample = np.transpose(np.array(lr_sample,dtype=np.float64),(2,0,1))
        sr_sample = Image.open(os.path.join(sr_name_path,image_names[i]))
        sr_tens = tensorify(np.array(sr_sample))
        sr_sample = np.transpose(np.array(sr_sample,dtype=np.float64),(2,0,1))
        src_sample = Image.open(os.path.join(src_name_path,image_names[i]))
        src_tens = tensorify(np.array(src_sample))
        src_sample = np.transpose(np.array(src_sample,dtype=np.float64),(2,0,1))


        lr_psnr.append(calculate_psnr(hr_sample,lr_sample))
        sr_psnr.append(calculate_psnr(hr_sample,sr_sample))
        src_psnr.append(calculate_psnr(hr_sample,src_sample))
        lr_ssim.append(ssim(hr_tens,lr_tens,val_range=255))
        sr_ssim.append(ssim(hr_tens,sr_tens,val_range=255))
        src_ssim.append(ssim(hr_tens,src_tens,val_range=255))

        del hr_sample,lr_sample,sr_sample,src_sample
        del hr_tens,lr_tens,sr_tens,src_tens

    sr_psnr,lr_psnr,src_psnr = np.array(sr_psnr),np.array(lr_psnr),np.array(src_psnr)
    sr_ssim,lr_ssim,src_ssim = np.array(sr_ssim),np.array(lr_ssim),np.array(src_ssim)
    print(mode)
    print(len(sr_psnr)," out of ",n," samples evaluated.")
    print("PSNR")
    print("SR       SRC       LR       #(SR>LR)")
    print(round(sr_psnr.mean(),3),round(src_psnr.mean(),3),round(lr_psnr.mean(),3),round((sr_psnr >= lr_psnr).sum(),3))
    print("-----")
    print("SSIM")
    print("SR     SRC     LR     #(SR>LR)")
    print(round(sr_ssim.mean(),3),round(src_ssim.mean(),3),round(lr_ssim.mean(),3),round((sr_ssim >= lr_ssim).sum(),3))
    print("\n____________\n")

    del sr_psnr,lr_psnr,sr_ssim,lr_ssim

test
133  out of  133  samples evaluated.
PSNR
SR       SRC       LR       #(SR>LR)
22.929 25.062 32.083 38
-----
SSIM
SR     SRC     LR     #(SR>LR)
0.781 0.797 0.692 86

____________



KeyboardInterrupt: 